In [ ]:
!pip install git+https://github.com/ssut/py-hanspell.git
!pip install konlpy

In [ ]:
#lib

import re

from hanspell import spell_checker
from konlpy.tag import Kkma

import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
import numpy as np

In [ ]:

text = '주문번호 1203'

text = re_sub(text)
text = check_spell(text)
text = kkma_tokenizer(text)
text = int_encode(text)
text = padding(text)


print(text)
pred(text)

In [ ]:
# 정규표현식 함수 정의

def re_sub(self):
    
    self = re.sub("\d\d\d\d"," @",self)
    self = re.sub("\d\d분"," #분",self)

    return self

In [ ]:
## 띄어쓰기, 맞춤법
def check_spell(self):
    
    spelled_sent = spell_checker.check(self)
    self = spelled_sent.checked

    return self

In [ ]:
# kkma 형태소 토큰화 함수정의
def kkma_tokenizer(self):
    kkma = Kkma()
    valid_pos = ['NNG','VV','SW','MAG']

    # tokenize
    token_text = kkma.pos(self)
    
    # 불용어 제거
    ls = []
    for token in token_text:
        
        if token[1] in valid_pos:
            ls.append(token[0])
        
    self = ls

    return self

In [ ]:
## df의 정수인코딩 column 생성 및 반영 함수화
# tokenizer.fit_on_texts 에서 특수문자 @,# 을 자체적으로 없앰, 이 부분 보완 필요

def int_encode(self):
    
    # 토큰 리스트 경로
    token_ls = pd.read_csv('/content/drive/MyDrive/Helricelus/token_ls.csv')['token']
    
    # 토크나이저 최적화
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(token_ls)

    seq = tokenizer.texts_to_sequences(self)

    ls = sum(seq,[])

    self = ls

    return self

In [ ]:
def padding(self):
    max_len = 8

    pad = [0]*(max_len-len(self))
    self = pad + self

    return self

In [ ]:
def pred(self):
    model = load_model('/content/drive/MyDrive/Helricelus/model/best_model.h5')
    
    self = np.array(self).reshape(-1,8)

    pred = model.predict(self)
    pred_result = np.argmax(pred)
    
    return pred_result